#### Library 


In [5]:
import gc
import json
import numpy as np 
import pandas as pd
import re
import tensorflow as tf 
import os

# add system path 

for dirname, _, filenames in os.walk('D:\kaggle\question answer competition'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm_notebook as tqdm
print(os.path)

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression

from scipy import spatial
import Levenshtein 

D:\kaggle\question answer competition\example\binary classification.ipynb
D:\kaggle\question answer competition\example\tf2_0_baseline_w_bert.py
D:\kaggle\question answer competition\example\using-tensorflow-2-0-w-bert-on-nq.ipynb
D:\kaggle\question answer competition\example\.ipynb_checkpoints\binary classification-checkpoint.ipynb
D:\kaggle\question answer competition\example\.ipynb_checkpoints\using-tensorflow-2-0-w-bert-on-nq-checkpoint.ipynb
D:\kaggle\question answer competition\example\__pycache__\tf2_0_baseline_w_bert.cpython-37.pyc
D:\kaggle\question answer competition\input\sample_submission.csv
D:\kaggle\question answer competition\input\simplified-nq-test.jsonl
D:\kaggle\question answer competition\input\simplified-nq-train.jsonl
D:\kaggle\question answer competition\input\tensorflow2-question-answering.zip
D:\kaggle\question answer competition\references\attention is all you need.pdf
D:\kaggle\question answer competition\references\discrete autoencoders for sequence models.

### test some frameworks 

In [10]:
stop_words = text.ENGLISH_STOP_WORDS.union(["book"])# what does this stop function mean ? 
# this returns a frozenset that can not be appended and deleted

### data preprocessing 
Extract features, constructor embedding vector for the question_context and answer_context 

In [24]:
training_data_path  = "D:\kaggle\question answer competition\input\simplified-nq-train.jsonl"
test_data_path = "D:\kaggle\question answer competition\input\simplified-nq-test.jsonl"
question_text_set   =  [] 
document_text_set = []
example_id_set = [] 

with open(test_data_path, 'r') as json_file:
    i=0 
    for line in json_file: 
        json_data = json.loads(line)
        question_text = json_data["question_text"]
        example_id = json_data["example_id"]
        document_text = json_data["document_text"]
        if(i<=10):
            question_text_set.append(question_text)
            document_text_set.append(document_text)
            example_id_set.append(example_id)
            
        i = i+1

print(document_text_set)
print(question_text_set)

for question in question_text_set: 
    for answer in answer_text_set: 
        
# prevent using the default setting for flgas 
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.compat.v1.app.flags.FLAGS)

flags = tf.compat.v1.app.flags


# clean funtion 
html_tags = ['<P>', '</P>', '<Table>', '</Table>', '<Tr>', '</Tr>', '<Ul>', '<Ol>', '<Dl>', '</Ul>', '</Ol>', \
             '</Dl>', '<Li>', '<Dd>', '<Dt>', '</Li>', '</Dd>', '</Dt>']
r_buf = ['is', 'are', 'do', 'does', 'did', 'was', 'were', 'will', 'can', 'the', 'a', 'an', 'of', 'in', 'and', 'on', \
         'what', 'where', 'when', 'which'] + html_tags

def clean(x):
    x = x.lower()
    for r in r_buf:
        x = x.replace(r, '')
    x = re.sub(' +', ' ', x)
    return x




['High Commission of South Africa , London - wikipedia <H1> High Commission of South Africa , London </H1> <Table> <Tr> <Th_colspan="2"> High Commission of South Africa in London </Th> </Tr> <Tr> <Td_colspan="2"> </Td> </Tr> <Tr> <Th> Location </Th> <Td> Trafalgar Square , London </Td> </Tr> <Tr> <Th> Address </Th> <Td> Trafalgar Square , London , WC2N 5DP </Td> </Tr> <Tr> <Th> Coordinates </Th> <Td> 51 ° 30 ′ 30 \'\' N 0 ° 07 ′ 37 \'\' W \ufeff / \ufeff 51.5082 ° N 0.1269 ° W \ufeff / 51.5082 ; - 0.1269 Coordinates : 51 ° 30 ′ 30 \'\' N 0 ° 07 ′ 37 \'\' W \ufeff / \ufeff 51.5082 ° N 0.1269 ° W \ufeff / 51.5082 ; - 0.1269 </Td> </Tr> <Tr> <Th> High Commissioner </Th> <Td> Vacant </Td> </Tr> </Table> Balcony of South Africa House <P> The High Commission of South Africa in London is the diplomatic mission from South Africa to the United Kingdom . It is located at South Africa House , a building on Trafalgar Square , London . As well as containing the offices of the High Commissioner , th


['who is the south african high commissioner in london', 'the office episode when they sing to michael', 'what is the main idea of the cross of gold speech', 'when was i want to sing in opera written', 'who does the voices in ice age collision course', 'who are the owners of the carolina hurricanes', 'what are the minds two tracks and what is dual processing', 'red cross got how many times nobel prize', 'when did they finish building the sydney opera house', 'where was the first sample of ascorbic acid isolated from', 'what episode of how i met your mother is the slap bet']


### feature expression     
project them on the different spaces 
